In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

In [ ]:
# import pyspark
# print(pyspark.__version__)
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [2]:
KAFKA_HOST = "broker:9092"  # "localhost:9092"
KAFKA_TOPIC = "cdc.public.transactions"

In [3]:
# Create a SparkSession

spark = SparkSession.builder.appName("KafkaStructuredStreaming").getOrCreate()

# spark = SparkSession.builder \
#     .appName("KafkaIntegration") \
#     .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
#     .getOrCreate()

In [4]:
# Define the schema for the "after" field in the JSON payload
after_schema = StructType(
    [
        StructField("transaction_id", StringType()),
        StructField("user_id", StringType()),
        StructField("timestamp", StringType()),
        StructField("amount", StringType()),
        StructField("currency", StringType()),
        StructField("city", StringType()),
        StructField("country", StringType()),
        StructField("merchant_name", StringType()),
        StructField("payment_method", StringType()),
        StructField("ip_address", StringType()),
        StructField("voucher_code", StringType()),
        StructField("affiliateid", StringType()),
        StructField("modified_by", StringType()),
        StructField("modified_at", StringType()),
        StructField("change_info", StringType()),
    ]
)

In [5]:
# Read the Kafka stream from the beginning
kafka_df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_HOST)
    .option("subscribe", KAFKA_TOPIC)
    .option("startingOffsets", "earliest")
    .load()
)

In [8]:
# Temporary query to display raw Kafka messages
raw_query = kafka_df.writeStream.outputMode("append").format("console").start()
raw_query.awaitTermination(timeout=20)  # Let it run for a short period, then stop

False

In [9]:
raw_query.stop()  # Stop the query to proceed with the rest of your codev

In [6]:
raw_query = kafka_df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("kafkaRawData") \
    .start()

# Then query the in-memory table after some processing time
spark.sql("SELECT * FROM kafkaRawData").show(truncate=False)

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



In [7]:
raw_query.stop()

In [ ]:
# Parse the JSON messages from Kafka and select the "after" field
parsed_df = kafka_df.select(
    from_json(col("value").cast("string"), after_schema).alias("after")
)

# Flatten the "after" structure into individual columns
flattened_df = parsed_df.select("after.*")

# Show the parsed DataFrame on the console (for demonstration purposes)
query = flattened_df.writeStream.outputMode("append").format("console").start()
query.awaitTermination()